In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, max_error
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor


In [2]:
df = pd.read_csv('train_data.csv', on_bad_lines='skip')

In [3]:
def prepro(df):
    df = pd.read_csv('train_data.csv', on_bad_lines='skip')
    df = df.drop(['BBLE','BLOCK','LOT','OWNER','BORO','EXCD1','STADDR','POSTCODE','AVLAND2','AVTOT2','EXLAND2','EXTOT2','EXCD2','PERIOD','VALTYPE','Community Board','Census Tract','BIN','NTA','New Georeferenced Column'], axis = 1)
    df['BLDGCL'] = np.where(df['BLDGCL'].isin({'A5', 'R3', 'B2', 'A1', 'C0', 'B9', 'A2', 'B1', 'A9', 'A0', 'C1', 'B3', 'V0', 'R1', 'C7', 'R2'}),
                           df['BLDGCL'], 'Other')
    df['TAXCLASS'] = df['TAXCLASS'].apply(lambda x: x[0:1])
    df = df.drop(df[df['FULLVAL'] == 0].index)
    y = df['FULLVAL']
    df = df.drop('FULLVAL', axis=1)
    df['YEAR'] = df['YEAR'].apply(lambda x: x[0:4])
    
    df = pd.get_dummies(df, columns=['EASEMENT','TAXCLASS','EXT','EXMPTCL','YEAR','Borough','BLDGCL'])
    
    
    
    imputer = KNNImputer(n_neighbors=1)
    imputer.fit(df)
    df = pd.DataFrame(imputer.transform(df.values), columns = df.columns, index = df.index)
    df = pd.get_dummies(df, columns=['Council District'])
    
    
    scaler = MinMaxScaler()
    scaler.fit(df)
    df = pd.DataFrame(scaler.transform(df.values), columns = df.columns, index = df.index)
    
    
    return df,y
    
    

In [4]:
df,y = prepro(df)

C:\Users\ltoma\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNNImputer was fitted with feature names
  warnings.warn(
C:\Users\ltoma\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [5]:
rf = RandomForestRegressor()
rf.fit(df,y)

RandomForestRegressor()

In [6]:
q = np.quantile(rf.feature_importances_, 0.50)
weak_columns = rf.feature_names_in_[np.where(rf.feature_importances_ < q)]
len(weak_columns)

38

In [7]:
df.drop(weak_columns, axis=1, inplace=True)

In [8]:
df.columns

Index(['LTFRONT', 'LTDEPTH', 'STORIES', 'AVLAND', 'AVTOT', 'EXLAND', 'EXTOT',
       'BLDFRONT', 'BLDDEPTH', 'Latitude', 'Longitude', 'TAXCLASS_1',
       'TAXCLASS_2', 'TAXCLASS_4', 'EXT_E', 'EXMPTCL_X1', 'EXMPTCL_X3',
       'YEAR_2017', 'YEAR_2018', 'Borough_BRONX', 'Borough_BROOKLYN',
       'Borough_MANHATTAN', 'Borough_STATEN IS', 'BLDGCL_A5', 'BLDGCL_B1',
       'BLDGCL_B3', 'BLDGCL_C1', 'BLDGCL_C7', 'BLDGCL_Other', 'BLDGCL_R1',
       'BLDGCL_V0', 'Council District_1.0', 'Council District_8.0',
       'Council District_14.0', 'Council District_17.0',
       'Council District_33.0', 'Council District_38.0',
       'Council District_51.0'],
      dtype='object')

In [9]:
X_train, X_test, y_train, y_test = train_test_split(df,y, test_size=0.3, random_state=42, shuffle=True)

In [10]:
from xgboost import XGBRegressor

C:\Users\ltoma\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [11]:
model = XGBRegressor()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mean_absolute_error(y_pred,y_test)

C:\Users\ltoma\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


173754.72891511364

In [12]:
y_pred = model.predict(X_train)
mean_absolute_error(y_pred,y_train)

C:\Users\ltoma\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


46608.92773407741

In [13]:
# from sklearn.ensemble import StackingRegressor
# from sklearn.ensemble import AdaBoostRegressor
# from sklearn.ensemble import GradientBoostingRegressor
# from sklearn.svm import SVC
# from sklearn.naive_bayes import GaussianNB
# from sklearn.linear_model import LinearRegression
# estimators = list()
# estimators.append(('rf', RandomForestRegressor()))
# estimators.append(('ada', AdaBoostRegressor()))
# estimators.append(('gboost', GradientBoostingRegressor()))
# estimators.append(('xgb', XGBRegressor(verbosity = 0)))
# f_estimator = LinearRegression()
# model10 = StackingRegressor(estimators=estimators, final_estimator=f_estimator)
# model10.fit(X_train, y_train)
# y_pred = model10.predict(X_test)
# mean_absolute_error(y_pred,y_test)

In [14]:
model = RandomForestRegressor()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mean_absolute_error(y_pred,y_test)

136113.61446245734

In [15]:
max_error(y_pred,y_test)

52397140.0

In [21]:
y_pred = model.predict(X_train)
mean_absolute_error(y_pred,y_train)

60153.02786079961